In [1]:
%pip install python-gnupg
%pip install eep153_tools
!pip install -r requirements.txt

  Using cached python_gnupg-0.5.2-py2.py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached eep153_tools-0.12.4-py2.py3-none-any.whl (4.9 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached CFEDemands-0.6.1-py2.py3-none-any.whl (45 kB)
  Using cached ConsumerDemands-0.4.2.dev0-py2.py3-none-any.whl (12 kB)
  Using cached ray-2.10.0-cp39-cp39-manylinux2014_x86_64.whl (65.1 MB)
  Using cached dvc-3.49.0-py3-none-any.whl (450 kB)
  Using cached xarray-2024.3.0-py3-none-any.whl (1.1 MB)
  Using cached rich-13.7.1-py3-none-any.whl (240 kB)
  Using cached iterative_telemetry-0.0.8-py3-none-any.whl (10 kB)
  Using cached dvc_studio_client-0.20.0-py3-none-any.whl (16 kB)
  Using cached grandalf-0.8-py3-none-any.whl (41 kB)
  Using cached dvc_objects-5.1.0-py3-none-any.whl (33 kB)
  Using cached configobj-5.0.8-py2.py3-none-any.whl (36 kB)
  Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Using cached ko

In [2]:
%pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install gspread_pandas

Note: you may need to restart the kernel to use updated packages.


<h2>[A] Getting the Data</h2>

In [4]:
from eep153_tools.sheets import read_sheets

url = 'https://docs.google.com/spreadsheets/d/1Ny8WWqxuPHirwHriHKNTCqG4l5V3GLQ4O6NIuGV1v_A/'

senegal_expenditures = read_sheets(url,sheet='Food Expenditures',nheaders=1)
senegal_expenditures.head()

,i,t,m,"Abats et tripes (foie, rognon, etc.)",Ail,Ananas,Arachide grillée,Arachides décortiquées ou pilées (noflaye),Arachides fraîches en coques,Arachides séchées en coques,...,Tomate fraîche,Tomate séchée,Viande d'autres volailles domestiques,Viande de b__uf,Viande de chameau,Viande de chèvre,Viande de mouton,Viande de porc,Viande de poulet,Vinaigre /moutarde
0,11011,2018,ziguinchor,NaN,600.0,NaN,NaN,NaN,NaN,NaN,...,300.0,NaN,NaN,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
1,110110,2018,ziguinchor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110111,2018,ziguinchor,NaN,NaN,NaN,500.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110112,2018,ziguinchor,NaN,NaN,NaN,100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,3000.0,NaN,NaN,NaN,NaN,NaN,50.0
4,11012,2018,ziguinchor,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
senegal_hh = read_sheets(url,sheet='Household Characteristics',nheaders=1)
senegal_hh.head()

,i,t,m,women,men,boys,girls,Males 00-03,Females 00-03,Males 04-08,...,Males 14-18,Females 14-18,Males 19-30,Females 19-30,Males 31-50,Females 31-50,Males 51-99,Females 51-99,log HSize,Rural
0,11011,2018,ziguinchor,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.000000,1
1,110110,2018,ziguinchor,1,1,1,1,0,0,0,...,0,1,1,0,0,1,0,0,1.386294,1
2,110111,2018,ziguinchor,0,2,2,0,0,0,0,...,2,0,1,0,1,0,0,0,1.386294,1
3,110112,2018,ziguinchor,2,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0.693147,1
4,11012,2018,ziguinchor,2,2,2,4,0,0,0,...,1,2,1,1,0,1,1,0,2.302585,1


# Place Holder

In [6]:
import pandas as pd
def clean_multiindex_df(df):
    # If the columns are a MultiIndex, join them with an underscore, otherwise just strip them
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(map(str, col)).strip() for col in df.columns.values]
    else:
        df.columns = [col.strip() for col in df.columns.tolist()]

    # Remove duplicated columns
    df = df.loc[:, ~df.columns.duplicated(keep='first')]

    # Drop columns that start with 'Unnamed'
    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)

    # Remove duplicated rows
    df = df.loc[~df.index.duplicated(), :]

    return df

# Clean the DataFrames with MultiIndex columns
z_cleaned = clean_multiindex_df(senegal_hh)
x_cleaned = clean_multiindex_df(senegal_expenditures)


In [7]:
x = senegal_expenditures

In [8]:
import numpy as np
if 'm' not in x.columns:
    x_cleaned['m'] = 1

x = x_cleaned.set_index(['i','t','m'])
x.columns.name = 'j'

x = x.apply(lambda x: pd.to_numeric(x,errors='coerce'))
x = x.replace(0,np.nan)

x

,,j,"Abats et tripes (foie, rognon, etc.)",Ail,Ananas,Arachide grillée,Arachides décortiquées ou pilées (noflaye),Arachides fraîches en coques,Arachides séchées en coques,"Arôme (Maggi, Jumbo, etc.)",Attiéke,"Aubergine, Courge/Courgette",...,Tomate fraîche,Tomate séchée,Viande d'autres volailles domestiques,Viande de b__uf,Viande de chameau,Viande de chèvre,Viande de mouton,Viande de porc,Viande de poulet,Vinaigre /moutarde
i,t,m,,,,,,,,,,,,,,,,,,,,,
11011,2018,ziguinchor,NaN,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,300.0,NaN,NaN,3000.0,NaN,NaN,NaN,NaN,NaN,NaN
110110,2018,ziguinchor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110111,2018,ziguinchor,NaN,NaN,NaN,500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110112,2018,ziguinchor,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3000.0,NaN,NaN,NaN,NaN,NaN,50.0
11012,2018,ziguinchor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985,2019,ziguinchor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2986,2019,ziguinchor,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,...,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3600.0,500.0
2987,2019,ziguinchor,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,...,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2800.0,500.0


In [9]:
# Get HH characteristics...
z = z_cleaned
if 'm' not in z.columns:
    z['m'] = 1

z = z.set_index(['i','t','m'])
z.columns.name = 'j'

z = z.apply(lambda x: pd.to_numeric(x,errors='coerce'))

z

,,j,women,men,boys,girls,Males 00-03,Females 00-03,Males 04-08,Females 04-08,Males 09-13,Females 09-13,Males 14-18,Females 14-18,Males 19-30,Females 19-30,Males 31-50,Females 31-50,Males 51-99,Females 51-99,log HSize,Rural
i,t,m,,,,,,,,,,,,,,,,,,,,
11011,2018,ziguinchor,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.000000,1
110110,2018,ziguinchor,1,1,1,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1.386294,1
110111,2018,ziguinchor,0,2,2,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,1.386294,1
110112,2018,ziguinchor,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0.693147,1
11012,2018,ziguinchor,2,2,2,4,0,0,0,1,1,1,1,2,1,1,0,1,1,0,2.302585,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985,2019,ziguinchor,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.000000,1
2986,2019,ziguinchor,5,1,1,5,0,0,0,2,1,0,0,4,0,3,0,1,1,0,2.484907,1
2987,2019,ziguinchor,1,4,2,0,0,0,1,0,1,0,0,0,3,0,0,1,1,0,1.945910,1


<h2>[A] Estimating Demand</h2>

In [10]:
import cfe

result = cfe.Regression(y=np.log(x.stack()),d=z)

result.get_beta().sort_values(ascending=False)

j
Huile de soja / végétale (e.g., Ninaal, Jaara, etc.)                   3.488494
Oignon frais                                                           2.844978
Riz local brisé                                                        2.838873
Lait en poudre                                                         2.763832
Mayonnaise                                                             2.703856
                                                                         ...   
Feuilles d'oseille (bissap)                                            0.789731
Autres tisanes et infusions n.d.a. (quinquelibat, citronelle, etc.)    0.761179
Poisson frais yaboy ou obo (sardinelle)                                0.740047
Pain moderne                                                           0.493078
Salade (laitue, batavia, etc.)                                         0.489491
Name: beta, Length: 63, dtype: float64

In [11]:
result.to_pickle('./foo.pickle')
result = cfe.read_pickle('foo.pickle')

In [ ]:
from cfe import regression as rgsn
import numpy as np

r = rgsn.read_pickle('foo.pickle')
xhat = result.predicted_expenditures()
p = np.exp(r.get_Ar())

In [ ]:
p.tail(20)